In [ ]:
import import_ipynb
import numpy as np
from scipy.sparse import lil_matrix
from fem_processing import master_domain, gaussian_quadrature

# Element “stiffness” matrix, $A_e$


$$
\mathbf{A}_e = \left( \int_{\hat{K}} a \left( J^{-1} \hat{\nabla} \hat{\varphi}_i \right) \cdot \left( J^{-1} \hat{\nabla} \hat{\varphi}_j \right) |J| \, d\hat{\mathbf{\xi}} \right) _{1 \leq \, i,j \, \leq \hat{N}}
\tag{1}
$$

# `global_nodes_coordinates()`

In [2]:
def global_nodes_coordinates(e, mesh_data):
    conn_list = [element['conn_list'] for element in mesh_data['conn_data']]
    node_list = [node['global_coord'] for node in mesh_data['nodes_data']]
    
    ai = [node_list[node - 1] for node in conn_list[e]]
    xi = [node[0] for node in ai]
    yi = [node[1] for node in ai]

    return ai, xi, yi

# `derivatives_at_master_domain()`

In [ ]:
def derivatives_at_master_domain(FINITE_ELEMENT, xik):
    type, order = FINITE_ELEMENT
    xi, eta = xik

    # Shape functions and derivatives
    if type == 'Triangle':
        if order == 1:
            varphi = master_domain.shape_functions_p1(xi, eta)
            dN_dxi, dN_deta = master_domain.derivatives_shape_functions_p1()
        elif order == 2:
            varphi = master_domain.shape_functions_p2(xi, eta)
            dN_dxi, dN_deta = master_domain.derivatives_shape_functions_p2(xi, eta)
        elif order == 3:
            varphi = master_domain.shape_functions_p3(xi, eta)
            dN_dxi, dN_deta = master_domain.derivatives_shape_functions_p3(xi, eta)
        else:
            raise ValueError('Invalid order for triangle element.')
        
    elif type == 'Quadrangle':
        if order == 1:
            varphi = master_domain.shape_functions_q1(xi, eta)
            dN_dxi, dN_deta = master_domain.derivatives_shape_functions_q1(xi, eta)
        elif order == 2:
            varphi = master_domain.shape_functions_q2(xi, eta)
            dN_dxi, dN_deta = master_domain.derivatives_shape_functions_q2(xi, eta)
        else:
            raise ValueError('Invalid order for quadrilateral element.')
    
    else:
        raise ValueError('Invalid element type.')
    
    # Verifica se as listas têm o mesmo tamanho
    if len(dN_dxi) != len(dN_deta):
        raise ValueError("As listas dN_dxi e dN_deta devem ter o mesmo tamanho.")

    Grad_phi = np.array([dN_dxi, dN_deta])

    return varphi, dN_dxi, dN_deta, Grad_phi

# `isomapping_to_global_coordinates()`

In [ ]:
def isomapping_to_global_coordinates(FINITE_ELEMENT, ai, xi):
    varphi, dN_dxi, dN_deta, GradN = derivatives_at_master_domain(FINITE_ELEMENT, xi)
    
    return sum(Ni * ai[i] for i, Ni in enumerate(varphi))

# `jacobian()` 

In [ ]:
def jacobian(FINITE_ELEMENT, mesh_data, cell, xik):
    Je = np.zeros((2, 2))

    # Get the element cell
    Ne = len(cell['conn'])

    # Get the global coordinates of the nodes
    xi = [mesh_data['nodes'][node]['xg'][0] for node in cell['conn']]
    yi = [mesh_data['nodes'][node]['xg'][1] for node in cell['conn']]

    # Shape functions and derivatives
    varphi, dN_dxi, dN_deta, grad_phi = derivatives_at_master_domain(FINITE_ELEMENT, xik)
    
    # Calcular o Jacobiano
    Je[0, 0] = sum(dN_dxi[i] * xi[i] for i in range(Ne))  # dx/dxi
    Je[0, 1] = sum(dN_dxi[i] * yi[i] for i in range(Ne))  # dy/dxi
    Je[1, 0] = sum(dN_deta[i] * xi[i] for i in range(Ne)) # dx/deta
    Je[1, 1] = sum(dN_deta[i] * yi[i] for i in range(Ne)) # dy/deta

    return Je

# `local_matrices()`

In [ ]:
def local_matrices(FINITE_ELEMENT, mesh_data, cell):
    # Get the element cell
    Ne = len(cell['conn'])

    # Initialize the local matrices
    Ae = np.zeros((Ne, Ne), dtype='complex128') # Matriz de rigidez do elemento
    Me = np.zeros((Ne, Ne), dtype='complex128') # Matriz de massa do elemento
    Pe = np.zeros((Ne, Ne), dtype='complex128') # Matriz de rigidez do contorno artificial
    Qe = np.zeros((Ne, Ne), dtype='complex128') # Matriz de massa do contorno artificial
    fe = np.zeros((Ne, 1))                      # Vetor de carga do elemento
    Je = np.zeros((2, 2))                       # Jacobiano do elemento

    # Get the Gauss points and weights
    gauss_points, gauss_weights = gaussian_quadrature.gauss_data(FINITE_ELEMENT)

    # Integração sobre o contorno Gamma_R
    gauss_points_1d = [
        (0.069432, 0.173928), (0.330009, 0.326072),
        (0.669991, 0.326072), (0.930568, 0.173928)
    ]
    
    for i, (xik, wk) in enumerate(zip(gauss_points, gauss_weights)):       
        # Material properties
        ka = cell['stiffness_term'][i]
        ma = cell['mass_term'][i]

        # Get the material properties
        rho = cell['source'][i]

        # Shape functions and derivatives
        varphi, _, _, grad_phi = derivatives_at_master_domain(FINITE_ELEMENT, xik)
        
        # Jacobian
        Je = jacobian(FINITE_ELEMENT, mesh_data, cell, xik) 
        
        # Determinante do Jacobiano
        Jdet, Jinv = np.abs(np.linalg.det(Je)), np.linalg.inv(Je)        
        
        # Matriz de rigidez
        Be = Jinv @ grad_phi
        Ae += Be.T @ ka @ Be * Jdet * wk

        # Matriz de massa
        Me += varphi.T * varphi * Jdet * wk * ma            
        
        # Element right-hand-side vector
        fe += varphi * Jdet * wk * rho

    # Integração sobre o contorno artificial Gamma_R
    if cell['abc']['type'] == 'BGT':
        # Get the global indices of the abc nodes
        abc_dict = cell['abc']['conn_idx']

        # Get the global coordinates of the abc nodes
        x1, x2 = [mesh_data['nodes'][idx]['xg'][0] for idx in abc_dict.values()]
        y1, y2 = [mesh_data['nodes'][idx]['xg'][1] for idx in abc_dict.values()]

        # Radius R (constant for the element)
        R = (np.sqrt(x1**2 + y1**2) + np.sqrt(x2**2 + y2**2)) / 2

        # Comprimento do elemento
        Je = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

        # Shape function derivatives (constant for linear elements)
        dphi_dl = [-1 / Je, 1 / Je]

        # Loop sobre os pontos de quadratura
        for (xi, wk) in gauss_points_1d:
            # Funções de forma no ponto xi
            phi = [(1 - xi), xi]

            # Compute q_ij matrix with weights, Jacobian, and R^2 factor
            for i, key_i in enumerate(abc_dict.keys()):
                for j, key_j in enumerate(abc_dict.keys()):
                    Pe[key_i, key_j] += wk * phi[i] * phi[j] * Je
                    Qe[key_i, key_j] += wk * R**2 * dphi_dl[i] * dphi_dl[j] * Je
            
    return Ae, fe, Me, Pe, Qe

# `global_matrices()`

In [ ]:
def global_matrices(FINITE_ELEMENT, mesh_data):
    # Inicializa a matriz global como uma matriz esparsa zero (tamanho NxN)
    Nn = len(mesh_data['nodes'])
    Ag = lil_matrix((Nn, Nn), dtype='complex128')
    Mg = lil_matrix((Nn, Nn), dtype='complex128')
    Pg = lil_matrix((Nn, Nn), dtype='complex128')
    Qg = lil_matrix((Nn, Nn), dtype='complex128')
    fg = lil_matrix((Nn, 1), dtype='complex128')

    # Início do processo de montagem
    for cell in mesh_data['cell'].values(): 
        # Get the cell's element number of nodes
        Ne = len(cell['conn'])

        # Compute the local data
        Ae, fe, Me, Pe, Qe = local_matrices(FINITE_ELEMENT, mesh_data, cell)

        # loop sobre os nós locais de cada elemento
        for i in range(Ne):  
            # Índice global do nó local i
            ig = cell['conn'][i]  
            
            # loop sobre os nós locais de cada elemento
            for j in range(Ne):  
                # nó global correspondente a j
                jg = cell['conn'][j]
                
                # preenche a matriz global A e M
                Ag[ig-1, jg-1] += Ae[i, j]
                Mg[ig-1, jg-1] += Me[i, j]
                Pg[ig-1, jg-1] += Pe[i, j]
                Qg[ig-1, jg-1] += Qe[i, j]  
            
            # preenche o vetor global b
            fg[ig-1, 0] += fe[i]

    return Ag, fg, Mg, Pg, Qg

# `apply_simple_dirichlet()`

In [ ]:
def apply_simple_dirichlet(Ag, fg, mesh_data):
    for key, node in mesh_data['nodes'].items():
        # Verifica se o nó é de Dirichlet
        if node['bc']['type'] == 'Dirichlet':
            e = int(key) - 1
            
            # Zera a linha correspondente ao nó de Dirichlet
            Ag[e, :] = 0
            
            # Define o valor 1 na diagonal para evitar singularidade
            Ag[e, e] = 1
            
            # Ajusta o vetor de forças com o valor de Dirichlet
            fg[e] = node['bc']['value']

    return Ag, fg

# `reduced_global_matrices()`

In [ ]:
def reduced_global_matrices(FINITE_ELEMENT, mesh_data):
    # Element connectivity, nodes and material properties
    nodes_data = mesh_data['nodes']

    # Get the mesh data
    free_nodes = {key: value for key, value in nodes_data.items() if value['bc']['type'] != 'Dirichlet'}

    # Mapeamento de índices globais para índices reduzidos
    global_to_reduced = {global_id: idx + 1 for idx, global_id in enumerate(free_nodes.keys())}

    # Initialize the global matrix and vector
    Agr = lil_matrix((len(free_nodes), len(free_nodes)), dtype='complex128')
    Mgr = lil_matrix((len(free_nodes), len(free_nodes)), dtype='complex128')
    Pgr = lil_matrix((len(free_nodes), len(free_nodes)), dtype='complex128')
    Qgr = lil_matrix((len(free_nodes), len(free_nodes)), dtype='complex128')
    fgr = lil_matrix((len(free_nodes), 1), dtype='complex128')

    # Início do processo de montagem
    for cell in mesh_data['cell'].values():
        # Get the cell's element number of nodes
        Ne = len(cell['conn'])

        # Compute the local data
        Ae, fe, Me, Pe, Qe = local_matrices(FINITE_ELEMENT, mesh_data, cell)

        # loop sobre os nós locais de cada elemento
        for i in range(Ne):  
            # Índice global do nó local i
            ig = cell['conn'][i]

            # Verifica se o nó é livre
            if ig in free_nodes:
                # Índice reduzido
                ig_red = global_to_reduced[ig] - 1
                
                # loop sobre os nós locais de cada elemento
                for j in range(Ne):
                    # Índice global do nó local j  
                    jg = cell['conn'][j]

                    # Verifica se o nó é livre
                    if jg in free_nodes:
                        # Índice reduzido
                        jg_red = global_to_reduced[jg] - 1

                        # preenche a matriz global A reduzida
                        Agr[ig_red, jg_red] += Ae[i, j]
                        Mgr[ig_red, jg_red] += Me[i, j]
                        Pgr[ig_red, jg_red] += Pe[i, j]
                        Qgr[ig_red, jg_red] += Qe[i, j]

                    # O nó global é de Dirichlet
                    else:
                        # Contribuição de Dirichlet para o vetor reduzido
                        fgr[ig_red, 0] += -Ae[i, j] * nodes_data[jg]['bc']['value']
                
                # preenche o vetor global b
                fgr[ig_red, 0] += fe[i]
    
    return Agr, fgr, Mgr, Pgr, Qgr

# `global_potentials_solution()`

In [ ]:
def global_potentials_solution(mesh_data, free_potentials):
    """
    Monta o vetor de potenciais globais V combinando potenciais calculados (livres) e prescritos (Dirichlet),
    utilizando o mapeamento global_to_reduced.

    Parâmetros:
    - mesh_data: Dados da malha, contendo nós e condições de contorno.
    - free_potentials: Potenciais calculados para os graus de liberdade livres (nós não-Dirichlet).

    Retorna:
    - V: Vetor contendo os potenciais globais de todos os nós.
    """
    # Número total de nós
    nodes_data = mesh_data['nodes']

    # Inicializa o vetor de potenciais globais com zeros
    uh = {}

    # Nós livres
    free_nodes = {key: value for key, value in nodes_data.items() if value['bc']['type'] != 'Dirichlet'}

    # Mapeamento de índices globais para índices reduzidos
    global_to_reduced = {global_id: idx + 1 for idx, global_id in enumerate(free_nodes.keys())}

    # Preenche o vetor V
    for key, node in nodes_data.items():
        # Verifica se o nó é livre
        if key in free_nodes:
            # Mapeamento global para reduzido
            reduced_index = global_to_reduced[key]
            uh[int(key)] = free_potentials[reduced_index - 1]

        # Nó de Dirichlet
        else:
            uh[int(key)] = node['bc']['value']

    return uh

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.

version 1.0. November, 2024.